# pytorch-tabnet pretrained model load
pytorch-tabnetの`clf.save_model()`はzipファイルで出力されます。しかし、Kaggleではzipファイルをアップロードすると、解凍して配置されてしまい、`clf.load_model()`が出来ないです。そこで、notebook内でbyte objectを作成、そこにzipファイルとしてファイルを保存するこで、読み込みを可能にしてます。

In pytorch-tabnet, `clf.save_model ()` is output as a zip file. However, in Kaggle, if you upload a zip file, it will be decompressed and placed.Therefore, `clf.load_model ()` cannot be done. So I create a byte object in the notebook and save the file as a zip file there.

In [ ]:
!pip install pytorch-tabnet

In [ ]:
import numpy as np
import pandas as pd
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.metrics import Metric
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score
import zipfile
import io

**io.BytesIO()でbyte objectを作成して、無理やりzipファイルを作る**

In [ ]:
tabnet_zip = io.BytesIO()
with zipfile.ZipFile(tabnet_zip, 'w') as z:
    z.write('../input/tabnet/model_params.json', arcname='model_params.json')
    z.write('../input/tabnet/network.pt', arcname='network.pt')

In [ ]:
clf = TabNetClassifier()
clf.load_model(tabnet_zip)

In [ ]:
test = pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv')
sample_submission = pd.read_csv('../input/tabular-playground-series-oct-2021/sample_submission.csv')

In [ ]:
test_arr = test.drop('id', axis=1).values
test_pred = clf.predict_proba(test_arr)

sample_submission['target'] = test_pred[:,1]
sample_submission.to_csv('tabnet_prediction.csv', index=False)